## Libraries

In [1]:
# Installing
!pip install gspread google-auth google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Importing
from datetime import datetime  # Date and time manipulation
from math import pi  # Mathematical constants
import re  # Regular expressions
import requests  # HTTP requests

import numpy as np  # Numerical operations
import pandas as pd  # Data analysis and manipulation

import gspread  # Google Sheets API
from google.oauth2.service_account import Credentials  # Authentication
import getpass  # Password management

## Functions

In [3]:
# Function for downloading files
def download_file(url, filename):
  response = requests.get(url)
  with open(filename, 'wb') as f:
    f.write(response.content)

In [4]:
# Function for initial file processing
def initial_processing(file_path):
  file_contents = open(file_path, encoding = 'utf-8').read() # read file
  if file_contents[0] == '#':
      file_contents = file_contents[1:] # remove first character
  else:
      print('First character not "#". URL:\n')
      print(f'{file_path}')
  lines = file_contents.split('\n') # split in lines
  if lines[1][0] == '#':
      second_line = lines.pop(1).strip()[1:] # store 2nd line without first character
  else:
      second_line = lines.pop(1).strip()
      print('Second line not starting with "#". URL:\n')
      print(f'{file_path}')
  file_contents = '\n'.join(lines) # restore file as a single string
  return file_contents, second_line

In [5]:
# Function for saving files
def save_file(file_path, file_contents):
  with open(file_path, 'w', encoding = 'utf-8') as f:
    f.write(file_contents)

In [6]:
# Function for analyzing a `DataFrame`
def analyze_df(df):
    # General quantities
    qty_rows = df.shape[0]
    qty_columns = df.shape[1]
    series_null_counts = df.isnull().sum()
    total_nulls = df.isnull().sum().sum()
    
    # Prepares a DataFrame for reporting
    df_report = pd.DataFrame(
        {
            'data_type': [df[col].dtype.name for col in df.columns],
            '#_unique': [df[col].nunique() for col in df.columns],
            '#_not_null': qty_rows - series_null_counts,
            '#_nulls': series_null_counts,
            '%_nulls': 100 * df.isnull().mean().round(4)
        }
    )

    print(df_report, '\n')
    print('Additionally:\n')
    print(f'The total number of null values is: {total_nulls}\n')
    print(f'The DataFrame has {qty_rows} rows and {qty_columns} columns.')

In [7]:
# Function for assigning datetime type
def vague_date_parser(date_str):
    """
    Note: this function is actually only capable of handling some cases
    of dates in the vague format described by Jonathan McDowell.
    Uncovered case examples: Quarters, Halfs, Centuries, Millenia.
    """
    formats = [
        '%Y %b %d %H%M:%S.%f',
        '%Y %b %d %H%M:%S',
        '%Y %b %d %H%M',
        '%Y %b %d',
        '%Y %b',
        '%Y',
    ]
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format = fmt)
        except ValueError:
            continue
    return pd.NaT

## Settings

In [8]:
# Limit removal for showing pandas.DataFrames' columns
pd.set_option('display.max_columns', None)
# Limit removal for showing pandas.DataFrames' rows
pd.set_option('display.max_rows', None)
# Modification of console width for displaying
pd.set_option('display.width', 150)

## Launch Tables

This section downloads the [Full List Launch Table](https://planet4589.org/space/gcat/web/launch/index.html) from Jonathan McDowell's website, creates a `DataFrame`, explores the data in its *raw* state, performs basic transformation, and, by filtering into another `DataFrame` and dropping columns, readies the data for later writing into a Google Sheets file.

### Loading

In [9]:
# Date/time format for table update lines
date_upd_format = 'Updated %Y %b %d %H%M:%S'
# Launch FList
url_lflist = 'https://planet4589.org/space/gcat/tsv/launch/launch.tsv'
# File paths
path_lflist = '06_lfulllist.tsv'
# File downloading
download_file(url_lflist, path_lflist)
# Initial file processing
file_lflist, update_lflist = initial_processing(path_lflist)
# Saving of initially processed files
save_file(path_lflist, file_lflist)
# DataFrames reading from initially processed files
df_lflist = pd.read_csv(path_lflist, sep = '\t')

In [10]:
# DataFrame's sneak peek
print(df_lflist.head())

   Launch_Tag   Launch_JD       Launch_Date LV_Type Variant Flight_ID Flight Mission FlightCode Platform Launch_Site Launch_Pad Ascent_Site  \
0  1942-A01    2430523.95  1942 Jun 13 1052     A-4       -         2      -       -          -        -         HVP         P7           -   
1  1942-A02    2430587.97  1942 Aug 16 1115     A-4       -         3      -       -          -        -         HVP         P7           -   
2  1942-S01    2430636.12  1942 Oct  3 1458     A-4       -         4      -       -          -        -         HVP         P7           -   
3  1942-A03    2430653.50       1942 Oct 21     A-4       -         5      -       -          -        -         HVP         P7           -   
4  1942-M01    2430672.50       1942 Nov  9     A-4       -         6      -       -          -        -         HVP         P7           -   

  Ascent_Pad   Apogee Apoflag  Range RangeFlag Dest  OrbPay Agency LaunchCode FailCode Group Category      LTCite        Cite Notes  
0      

In [11]:
# DataFrames' sneak peek
print(df_lflist.tail())

      Launch_Tag   Launch_JD          Launch_Date         LV_Type Variant          Flight_ID        Flight               Mission  FlightCode  \
75121  2025-120   2460831.49  2025 Jun  4 2340:30        Falcon 9     FT5  xxx/B1063.26 F486  Starlink-272  Starlink Group 11-22  V2MO 11-22   
75122  2025-121   2460832.36     2025 Jun  5 2045  Chang Zheng 6A       -                 Y8       WHWD 04               WHWD 04           -   
75123  2025-122   2460833.70     2025 Jun  7 0454        Falcon 9     FT5  xxx/B1085.8  F487        SXM-10                SXM-10           -   
75124  2025-123   2460835.10  2025 Jun  8 1420:10        Falcon 9     FT5  xxx/B1088.7  F488  Starlink-273   Starlink Group 15-8   V2MO 15-8   
75125  2025-124   2460837.05  2025 Jun 10 1305:40        Falcon 9     FT5  xxx/B1083.12 F489  Starlink-274  Starlink Group 12-24  V2MO 12-24   

      Platform Launch_Site Launch_Pad Ascent_Site Ascent_Pad   Apogee Apoflag  Range RangeFlag Dest  OrbPay Agency LaunchCode FailCode 

In [12]:
# Obtain and check update date and time
dt_upd_lflist = datetime.strptime(update_lflist.strip(), date_upd_format)
print(dt_upd_lflist)

2025-06-11 12:45:27


### Preliminary EDA

#### Null assignment

In [13]:
# Iterate over all column names, if they are type 'object', remove starting
# and trailing blanks
for col in df_lflist.columns:
    if df_lflist[col].dtype.name == 'object':
        df_lflist.loc[:, col] = df_lflist[col].apply(lambda x: str(x)).apply(lambda x: x.strip())

# For convenience, assign <NA> values to work before writing into Google Sheets
df_lflist = df_lflist.map(lambda x: pd.NA if isinstance(x, str) and x == '-' else x)

# DataFrame's analysis
analyze_df(df_lflist)

            data_type  #_unique  #_not_null  #_nulls  %_nulls
Launch_Tag     object     75126       75126        0     0.00
Launch_JD     float64     60961       75126        0     0.00
Launch_Date    object     64669       75126        0     0.00
LV_Type        object      1533       75126        0     0.00
Variant        object       118        3489    71637    95.36
Flight_ID      object     26549       32043    43083    57.35
Flight         object     21545       53334    21792    29.01
Mission        object      7946       13010    62116    82.68
FlightCode     object      2291        3264    71862    95.66
Platform       object       335        6193    68933    91.76
Launch_Site    object       663       75126        0     0.00
Launch_Pad     object      1061       24703    50423    67.12
Ascent_Site    object        38         767    74359    98.98
Ascent_Pad     object        57         750    74376    99.00
Apogee         object      2924       74440      686     0.91
Apoflag 

#### EDA

In [14]:
# Descriptive statistics for all columns
df_lflist.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Launch_Tag,75126,75126,1942-A01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Launch_JD,75126.0,NaN,NaN,NaN,2443251.515405,5500.966772,2430523.95,2440229.235,2442446.715,2445117.32,2460837.05
Launch_Date,75126,64669,1968,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LV_Type,75126,1533,Rocketsonde,21362,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Variant,3489,118,?,1226,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Flight_ID,32043,26549,MR-12,358,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Flight,53334,21545,"Starute,Datasonde",8771,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mission,13010,7946,R&D,256,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FlightCode,3264,2291,MGCh 8F126,214,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Platform,6193,335,SHIR,541,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Transforming

#### Datetime columns

In [15]:
# Removal of trailing '?' (accuracy flag) into new column
df_lflist['LD_Acc'] = pd.NA

for idx, value in zip(df_lflist.index, df_lflist['Launch_Date']):
    if value[-1] == '?':
        df_lflist.loc[idx, 'LD_Acc'] = df_lflist.loc[idx, 'Launch_Date'][-1] # Assign last character
        df_lflist.loc[idx, 'Launch_Date'] = value[:-1]  # Remove the trailing '?'

# Removal of quarter indications 'Q#' into new column
df_lflist['LD_Qrt'] = pd.NA

for idx, value in zip(df_lflist.index, df_lflist['Launch_Date']):
    if re.search(r'Q\d$', value):
        df_lflist.loc[idx, 'LD_Qrt'] = df_lflist.loc[idx, 'Launch_Date'][-2:] # Assign last two characters
        df_lflist.loc[idx, 'Launch_Date'] = value[:-2].strip() # Remove last characters

# Print the first 20 rows of the relevant columns
print(df_lflist[['Launch_Date', 'LD_Acc', 'LD_Qrt']].head(20))

         Launch_Date LD_Acc LD_Qrt
0   1942 Jun 13 1052   <NA>   <NA>
1   1942 Aug 16 1115   <NA>   <NA>
2   1942 Oct  3 1458   <NA>   <NA>
3        1942 Oct 21   <NA>   <NA>
4        1942 Nov  9   <NA>   <NA>
5        1942 Nov 28   <NA>   <NA>
6        1942 Dec 12   <NA>   <NA>
7        1943 Jan  7   <NA>   <NA>
8        1943 Jan 25   <NA>   <NA>
9        1943 Feb 17   <NA>   <NA>
10       1943 Feb 19   <NA>   <NA>
11       1943 Mar  3   <NA>   <NA>
12       1943 Mar 18   <NA>   <NA>
13       1943 Mar 25   <NA>   <NA>
14       1943 Apr 14   <NA>   <NA>
15  1943 Apr 22 1425   <NA>   <NA>
16       1943 May 14   <NA>   <NA>
17  1943 May 26 1100   <NA>   <NA>
18  1943 May 26 1630      ?   <NA>
19       1943 May 27   <NA>   <NA>


In [16]:
# Review of values in new columns
print("How many '?' flags?")
print(df_lflist['LD_Acc'].value_counts(dropna = False))
print()
print("How many of each quarter indications?")
print(df_lflist['LD_Qrt'].value_counts(dropna = False))

How many '?' flags?
LD_Acc
<NA>    72922
?        2204
Name: count, dtype: int64

How many of each quarter indications?
LD_Qrt
<NA>    75031
Q3         51
Q2         19
Q4         17
Q1          8
Name: count, dtype: int64


In [17]:
# Data type conversion
df_lflist['Pars_L_Date'] = df_lflist['Launch_Date'].apply(vague_date_parser)
# Extraction of date&time as string
df_lflist['Pars_LD_str'] = df_lflist['Pars_L_Date'].dt.strftime('%Y%m%d%H%M%S.%f').replace(np.nan, '')
df_lflist['Pars_LDay_str'] = df_lflist['Pars_L_Date'].dt.strftime('%Y%m%d').replace(np.nan, '')

# DataFrame's analysis
analyze_df(df_lflist)

                    data_type  #_unique  #_not_null  #_nulls  %_nulls
Launch_Tag             object     75126       75126        0     0.00
Launch_JD             float64     60961       75126        0     0.00
Launch_Date            object     64343       75126        0     0.00
LV_Type                object      1533       75126        0     0.00
Variant                object       118        3489    71637    95.36
Flight_ID              object     26549       32043    43083    57.35
Flight                 object     21545       53334    21792    29.01
Mission                object      7946       13010    62116    82.68
FlightCode             object      2291        3264    71862    95.66
Platform               object       335        6193    68933    91.76
Launch_Site            object       663       75126        0     0.00
Launch_Pad             object      1061       24703    50423    67.12
Ascent_Site            object        38         767    74359    98.98
Ascent_Pad          

#### Working DataFrame

In [18]:
# DataFrame with all launches since 2000
# Selected years list
lst_sel_years = list(range(2000,2026))
# DataFrame from filtering by years and specific launch codes
df_lflist2000 = df_lflist[
    (df_lflist['Pars_L_Date'].dt.year.isin(lst_sel_years))
    & (df_lflist['LaunchCode'].str[0].isin(list('OSDTY')))
].reset_index(drop = True)
print(df_lflist2000.head(10))
print()
print("Shape:")
print(df_lflist2000.shape)

  Launch_Tag   Launch_JD          Launch_Date         LV_Type Variant            Flight_ID               Flight       Mission FlightCode Platform  \
0   2000-S01  2451553.37     2000 Jan  9 2050           S-310    <NA>             S-310-29             WAVE2000          <NA>       <NA>     <NA>   
1   2000-W01  2451555.58  2000 Jan 12 0200:00         MT-135P    <NA>                 <NA>                 <NA>          <NA>       <NA>     <NA>   
2   2000-W02  2451562.58  2000 Jan 19 0200:00         MT-135P    <NA>                 <NA>                 <NA>          <NA>       <NA>     <NA>   
3   2000-001  2451564.54  2000 Jan 21 0103:01       Atlas IIA    <NA>  AC-138 MLV-8/IABS-9  USA 148 (DSCS 3B-8)  DSCS III B-8      MLV-8     <NA>   
4   2000-W03  2451565.18     2000 Jan 21 1622      Super Loki    <NA>             LiteStar                 <NA>          <NA>       <NA>     <NA>   
5   2000-002  2451568.54     2000 Jan 25 0104      Ariane 42L   42L-3            V126 L494           Galax

In [19]:
# Some explicit modifications
# These fill missing data or ensure data consistency
df_lflist2000.loc[df_lflist2000['Launch_Tag'] == '2024-049', 'FlightCode'] = "V2M 6-44 (23 Ku)"
df_lflist2000.loc[df_lflist2000['Agency'] == 'ZHSH/SAST', 'Agency'] = "SAST"
df_lflist2000.loc[df_lflist2000['Agency'] == 'ISRO/NSIL', 'Agency'] = "ISRO"

In [20]:
# DataFrame backup
df_lflist2000_bkp = df_lflist2000.copy()

# Replacement of null values by empty string
# This is needed to avoid errors when writing into Google Sheets
df_lflist2000 = df_lflist2000.where(pd.notnull(df_lflist2000), '')

# Removal of unnecessary columns
df_lflist2000 = df_lflist2000.drop(
    columns = [
        'Launch_JD',
        'Pars_L_Date',
        'Launch_Date',
        'LTCite',
        'Cite',
        'Notes',
        'LD_Acc',
        'LD_Qrt',
    ]
)

# DataFrame's analysis
analyze_df(df_lflist2000)

              data_type  #_unique  #_not_null  #_nulls  %_nulls
Launch_Tag       object      3975        3975        0      0.0
LV_Type          object       353        3975        0      0.0
Variant          object        55        3975        0      0.0
Flight_ID        object      2907        3975        0      0.0
Flight           object      3463        3975        0      0.0
Mission          object      2497        3975        0      0.0
FlightCode       object       730        3975        0      0.0
Platform         object        19        3975        0      0.0
Launch_Site      object       100        3975        0      0.0
Launch_Pad       object       146        3975        0      0.0
Ascent_Site      object         7        3975        0      0.0
Ascent_Pad       object        16        3975        0      0.0
Apogee           object      1455        3975        0      0.0
Apoflag          object         2        3975        0      0.0
Range            object        15       

## Organizations Table

This section downloads the [Space Organizations Table](https://planet4589.org/space/gcat/web/orgs/index.html) from Jonathan McDowell's website, creates a `DataFrame`, explores the data in its *raw* state, performs basic transformations, and, by dropping columns, readies the data for later writing into a Google Sheets file.

### Loading

In [21]:
# Date/time format for table update lines
date_upd_format = 'Updated %Y %b %d %H%M:%S'
# Organizations List
url_oglist = 'https://planet4589.org/space/gcat/tsv/tables/orgs.tsv'
# File paths
path_oglist = '02_orgslist.tsv'
# File downloading
download_file(url_oglist, path_oglist)
# Initial file processing
file_oglist, update_oglist = initial_processing(path_oglist)
# Saving of initially processed files
save_file(path_oglist, file_oglist)
# DataFrames reading from initially processed files
df_oglist = pd.read_csv(path_oglist, sep = '\t')

# DataFrames' sneak peek
print(df_oglist.head())
print()
print("Shape:")
print(df_oglist.shape)

    Code  UCode StateCode Type Class    TStart TStop       ShortName                                    Name                               Location  \
0  EARTH  EARTH     EARTH   AP     C         -     -           Earth                                   Earth                                  Earth   
1   LUNA   LUNA      LUNA   AP     C         -     -            Luna                                    Luna                                   Luna   
2   SSYS   SSYS      SSYS   AP     C         -     -    Solar System  Solar System natural celestial objects                                    Sol   
3    AAT    AAT       AAT  CYP     C      1841     -  Aus. Antarctic          Australian Antarctic Territory  Davis Staton, Princess Elizabeth Land   
4    ADG    ADG        RU  CYP     C  1991 Jan     -          Adygea                      Respublika Adygeya                                 Maykop   

       Longitude     Latitude  Error Parent     ShortEName                                   

In [22]:
# Obtain and check update date and time
dt_upd_oglist = datetime.strptime(update_oglist.strip(), date_upd_format)
print(dt_upd_oglist)

2025-06-11 12:45:11


### Preliminary EDA

#### Null assignment

In [23]:
# Iterate over all column names, if they are type 'object', remove starting
# and trailing blanks
for col in df_oglist.columns:
    if df_oglist[col].dtype.name == 'object':
        df_oglist.loc[:, col] = df_oglist[col].apply(lambda x: str(x)).apply(lambda x: x.strip())

# For convenience, assign <NA> values to work before writing into Google Sheets
df_oglist = df_oglist.map(lambda x: pd.NA if isinstance(x, str) and x == '-' else x)

# DataFrame's analysis
analyze_df(df_oglist)

           data_type  #_unique  #_not_null  #_nulls  %_nulls
Code          object      3872        3872        0     0.00
UCode         object      2888        3872        0     0.00
StateCode     object       186        3872        0     0.00
Type          object        97        3519      353     9.12
Class         object         4        3872        0     0.00
TStart        object       984        3598      274     7.08
TStop         object       523        1357     2515    64.95
ShortName     object      3619        3871        1     0.03
Name          object      3794        3871        1     0.03
Location      object      1619        3865        7     0.18
Longitude     object      1395        3869        3     0.08
Latitude      object      1216        3869        3     0.08
Error        float64        13        3872        0     0.00
Parent        object       458        1280     2592    66.94
ShortEName    object       381         404     3468    89.57
EName         object    

#### EDA

In [24]:
# Descriptive statistics for all columns
df_oglist.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Code,3872,3872,EARTH,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UCode,3872,2888,GUKOS,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StateCode,3872,186,US,1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,3519,97,O,979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Class,3872,4,B,1808,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TStart,3598,984,2015,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TStop,1357,523,*,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ShortName,3871,3619,General Atomics,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,3871,3794,Sea Launch AG,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Location,3865,1619,Moskva,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Transforming

#### Datetime columns

In [25]:
# Removal of trailing '?' (accuracy flag) into new columns
df_oglist['TStart_Acc'] = pd.NA
df_oglist['TStop_Acc'] = pd.NA

for idx, value_i, value_f in zip(df_oglist.index, df_oglist['TStart'], df_oglist['TStop']):
    if isinstance(value_i, str) and value_i.endswith('?'):
        df_oglist.loc[idx, 'TStart_Acc'] = value_i[-1] # Assign last character
        df_oglist.loc[idx, 'TStart'] = value_i[:-1]  # Remove the trailing '?'

    if isinstance(value_f, str) and value_f.endswith('?'):
        df_oglist.loc[idx, 'TStop_Acc'] = value_f[-1] # Assign last character
        df_oglist.loc[idx, 'TStop'] = value_f[:-1]  # Remove the trailing '?'

# Removal of quarter indications 'Q#' into new columns
df_oglist['TStart_Qrt'] = pd.NA
df_oglist['TStop_Qrt'] = pd.NA

for idx, value_i, value_f in zip(df_oglist.index, df_oglist['TStart'], df_oglist['TStop']):
    if isinstance(value_i, str) and re.search(r'Q\d$', value_i):
        df_oglist.loc[idx, 'TStart_Qrt'] = value_i[-2:] # Assign last two characters
        df_oglist.loc[idx, 'TStart'] = value_i[:-2].strip() # Remove quarter indication

    if isinstance(value_f, str) and re.search(r'Q\d$', value_f):
        df_oglist.loc[idx, 'TStop_Qrt'] = value_f[-2:] # Assign last two characters
        df_oglist.loc[idx, 'TStop'] = value_f[:-2].strip() # Remove quarter indication

# Print the first 20 rows of the relevant columns
print(df_oglist[['TStart', 'TStart_Acc', 'TStart_Qrt', 'TStop', 'TStop_Acc', 'TStop_Qrt']].head(20))

         TStart TStart_Acc TStart_Qrt     TStop TStop_Acc TStop_Qrt
0          <NA>       <NA>       <NA>      <NA>      <NA>      <NA>
1          <NA>       <NA>       <NA>      <NA>      <NA>      <NA>
2          <NA>       <NA>       <NA>      <NA>      <NA>      <NA>
3          1841       <NA>       <NA>      <NA>      <NA>      <NA>
4      1991 Jan       <NA>       <NA>      <NA>      <NA>      <NA>
5          2004       <NA>       <NA>      <NA>      <NA>      <NA>
6          1981       <NA>       <NA>      <NA>      <NA>      <NA>
7          1832       <NA>       <NA>      1981      <NA>      <NA>
8          1991       <NA>       <NA>      <NA>      <NA>      <NA>
9      1978 Apr       <NA>       <NA>      <NA>      <NA>      <NA>
10         1954       <NA>       <NA>  2010 Oct      <NA>      <NA>
11         1975       <NA>       <NA>      <NA>      <NA>      <NA>
12         <NA>       <NA>       <NA>      <NA>      <NA>      <NA>
13  1816 Jul  9       <NA>       <NA>      <NA> 

In [26]:
# Partial DataFrame's Analysis
analyze_df(df_oglist[['TStart', 'TStart_Acc', 'TStart_Qrt', 'TStop', 'TStop_Acc', 'TStop_Qrt']])

           data_type  #_unique  #_not_null  #_nulls  %_nulls
TStart        object       912        3598      274     7.08
TStart_Acc    object         1         279     3593    92.79
TStart_Qrt    object         0           0     3872   100.00
TStop         object       485        1357     2515    64.95
TStop_Acc     object         1          96     3776    97.52
TStop_Qrt     object         0           0     3872   100.00 

Additionally:

The total number of null values is: 17902

The DataFrame has 3872 rows and 6 columns.


In [27]:
# Data type conversion
df_oglist['Pars_TStart'] = df_oglist['TStart'].apply(vague_date_parser)
df_oglist['Pars_TStop'] = df_oglist['TStop'].apply(vague_date_parser)
# Extraction of date&time as string
df_oglist['Pars_TStart_str'] = df_oglist['Pars_TStart'].dt.strftime('%Y%m%d%H%M%S.%f').replace(np.nan, '')
df_oglist['Pars_TStop_str'] = df_oglist['Pars_TStop'].dt.strftime('%Y%m%d%H%M%S.%f').replace(np.nan, '')

# DataFrame's analysis
analyze_df(df_oglist)

                      data_type  #_unique  #_not_null  #_nulls  %_nulls
Code                     object      3872        3872        0     0.00
UCode                    object      2888        3872        0     0.00
StateCode                object       186        3872        0     0.00
Type                     object        97        3519      353     9.12
Class                    object         4        3872        0     0.00
TStart                   object       912        3598      274     7.08
TStop                    object       485        1357     2515    64.95
ShortName                object      3619        3871        1     0.03
Name                     object      3794        3871        1     0.03
Location                 object      1619        3865        7     0.18
Longitude                object      1395        3869        3     0.08
Latitude                 object      1216        3869        3     0.08
Error                   float64        13        3872        0  

#### Working DataFrame

In [28]:
# DataFrame's sneak peek
df_oglist.head(1)

,Code,UCode,StateCode,Type,Class,TStart,TStop,ShortName,Name,Location,Longitude,Latitude,Error,Parent,ShortEName,EName,UName,TStart_Acc,TStop_Acc,TStart_Qrt,TStop_Qrt,Pars_TStart,Pars_TStop,Pars_TStart_str,Pars_TStop_str
0,EARTH,EARTH,EARTH,AP,C,<NA>,<NA>,Earth,Earth,Earth,<NA>,<NA>,0.02,<NA>,Earth (Terra),Earth,Earth,<NA>,<NA>,<NA>,<NA>,NaT,NaT,,


In [29]:
# DataFrame backup
df_oglist_bkp = df_oglist.copy()

# Replacement of null values by empty string
# This is needed to avoid errors when writing into Google Sheets
df_oglist = df_oglist.where(pd.notnull(df_oglist), '')

# Removal of unnecessary columns
df_oglist = df_oglist.drop(
    columns = [
        'TStart',
        'TStop',
        'Location',
        'Longitude',
        'Latitude',
        'Error',
        'Parent',
        'TStart_Acc',
        'TStop_Acc',
        'TStart_Qrt',
        'TStop_Qrt',
        'Pars_TStart',
        'Pars_TStop'
    ]
)

# DataFrame's analysis
analyze_df(df_oglist)

                data_type  #_unique  #_not_null  #_nulls  %_nulls
Code               object      3872        3872        0      0.0
UCode              object      2888        3872        0      0.0
StateCode          object       186        3872        0      0.0
Type               object        98        3872        0      0.0
Class              object         4        3872        0      0.0
ShortName          object      3620        3872        0      0.0
Name               object      3795        3872        0      0.0
ShortEName         object       382        3872        0      0.0
EName              object      1237        3872        0      0.0
UName              object      3746        3872        0      0.0
Pars_TStart_str    object       783        3872        0      0.0
Pars_TStop_str     object       426        3872        0      0.0 

Additionally:

The total number of null values is: 0

The DataFrame has 3872 rows and 12 columns.


## Launch Sites Table

This section downloads the [Launch Sites Table](https://planet4589.org/space/gcat/web/sites/index.html) from Jonathan McDowell's website, creates a `DataFrame`, explores the data in its *raw* state, performs basic transformations, and, by dropping columns, readies the data for later writing into a Google Sheets file.

### Loading

In [30]:
# Date/time format for table update lines
date_upd_format = 'Updated %Y %b %d %H%M:%S'
# Launch Sites List
url_lslist = 'https://planet4589.org/space/gcat/tsv/tables/sites.tsv'
# File paths
path_lslist = '03_lsitlist.tsv'
# File downloading
download_file(url_lslist, path_lslist)
# Initial file processing
file_lslist, update_lslist = initial_processing(path_lslist)
# Saving of initially processed files
save_file(path_lslist, file_lslist)
# DataFrames reading from initially processed files
df_lslist = pd.read_csv(path_lslist, sep = '\t')

# DataFrames' sneak peek
print(df_lslist.head())
print()
print("Shape:")
print(df_lslist.shape)

    Site Code  UCode Type StateCode       TStart        TStop ShortName                                               Name  \
0  OCEAN    -  OCEAN   LS     EARTH            -            -     Ocean                                 Ocean launch sites   
1    A51    -    A51   LS        US         1968         1972   Area 51         Groom Lake Air Force Base, Area 51, Nevada   
2   AAAF    -   HADC   LS        US  1941 Apr 13  1949 Dec  2      AAAF                          Alamagordo Army Air Field   
3   ABER    -   ABER   LS        UK  1959 Jun 29  1972 Jul 20   ABER  -                                    Aberporth Range   
4  AFMDC    -   HADC   LS        US  1957 Sep  1  1970 Aug  1     AFMDC  Air Force Missile Development Center, Holloman...   

                     Location      Longitude     Latitude   Error Parent ShortEName EName Group                                              UName  
0                           -              -            -  0.0000      -      Ocean     -     

In [31]:
# Obtain and check update date and time
dt_upd_lslist = datetime.strptime(update_lslist.strip(), date_upd_format)
print(dt_upd_lslist)

2025-06-11 12:45:11


### Preliminary EDA

#### Null assignment

In [32]:
# Iterate over all column names, if they are type 'object', remove starting
# and trailing blanks
for col in df_lslist.columns:
    if df_lslist[col].dtype.name == 'object':
        df_lslist.loc[:, col] = df_lslist[col].apply(lambda x: str(x)).apply(lambda x: x.strip())

# For convenience, assign <NA> values to work before writing into Google Sheets
df_lslist = df_lslist.map(lambda x: pd.NA if isinstance(x, str) and x == '-' else x)

# DataFrame's analysis
analyze_df(df_lslist)

           data_type  #_unique  #_not_null  #_nulls  %_nulls
Site          object       703         703        0     0.00
Code          object         0           0      703   100.00
UCode         object       669         703        0     0.00
Type          object         3         703        0     0.00
StateCode     object        82         703        0     0.00
TStart        object       368         638       65     9.25
TStop         object       322         556      147    20.91
ShortName     object       512         703        0     0.00
Name          object       699         703        0     0.00
Location      object       112         336      367    52.20
Longitude     object       492         680       23     3.27
Latitude      object       468         680       23     3.27
Error        float64        30         703        0     0.00
Parent        object       141         701        2     0.28
ShortEName    object         8          14      689    98.01
EName         object    

#### EDA

In [33]:
# Descriptive statistics for all columns
df_lslist.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Site,703,703,OCEAN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Code,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UCode,703,669,CC,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,703,3,LS,453,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StateCode,703,82,SU,239,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TStart,638,368,2022,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TStop,556,322,*,111,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ShortName,703,512,SHIR,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,703,699,"Cape Canaveral Air Force Station, Eastern Test...",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Location,336,112,"Minia, Atlantic Ocean",65,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Transforming

#### Datetime columns

In [34]:
# Removal of trailing '?' (accuracy flag) into new columns
df_lslist['TStart_Acc'] = pd.NA
df_lslist['TStop_Acc'] = pd.NA

for idx, value_i, value_f in zip(df_lslist.index, df_lslist['TStart'], df_lslist['TStop']):
    if isinstance(value_i, str) and value_i.endswith('?'):
        df_lslist.loc[idx, 'TStart_Acc'] = value_i[-1] # Assign last character
        df_lslist.loc[idx, 'TStart'] = value_i[:-1]  # Remove the trailing '?'

    if isinstance(value_f, str) and value_f.endswith('?'):
        df_lslist.loc[idx, 'TStop_Acc'] = value_f[-1] # Assign last character
        df_lslist.loc[idx, 'TStop'] = value_f[:-1]  # Remove the trailing '?'

# Removal of quarter indications 'Q#' into new columns
df_lslist['TStart_Qrt'] = pd.NA
df_lslist['TStop_Qrt'] = pd.NA

for idx, value_i, value_f in zip(df_lslist.index, df_lslist['TStart'], df_lslist['TStop']):
    if isinstance(value_i, str) and re.search(r'Q\d$', value_i):
        df_lslist.loc[idx, 'TStart_Qrt'] = value_i[-2:] # Assign last two characters
        df_lslist.loc[idx, 'TStart'] = value_i[:-2].strip() # Remove quarter indication

    if isinstance(value_f, str) and re.search(r'Q\d$', value_f):
        df_lslist.loc[idx, 'TStop_Qrt'] = value_f[-2:] # Assign last two characters
        df_lslist.loc[idx, 'TStop'] = value_f[:-2].strip() # Remove quarter indication

# Print the first 20 rows of the relevant columns
print(df_lslist[['TStart', 'TStart_Acc', 'TStart_Qrt', 'TStop', 'TStop_Acc', 'TStop_Qrt']].head(20))

         TStart TStart_Acc TStart_Qrt        TStop TStop_Acc TStop_Qrt
0          <NA>       <NA>       <NA>         <NA>      <NA>      <NA>
1          1968       <NA>       <NA>         1972      <NA>      <NA>
2   1941 Apr 13       <NA>       <NA>  1949 Dec  2      <NA>      <NA>
3   1959 Jun 29       <NA>       <NA>  1972 Jul 20      <NA>      <NA>
4   1957 Sep  1       <NA>       <NA>  1970 Aug  1      <NA>      <NA>
5   1951 Jun 30       <NA>       <NA>  1955 Dec 16      <NA>      <NA>
6          1962       <NA>       <NA>         1964      <NA>      <NA>
7          1984       <NA>       <NA>         <NA>      <NA>      <NA>
8   1956 Sep 24       <NA>       <NA>  1962 May 24      <NA>      <NA>
9          2021       <NA>       <NA>         <NA>      <NA>      <NA>
10  1994 Aug 19       <NA>       <NA>            *      <NA>      <NA>
11        1960s       <NA>       <NA>        1990s      <NA>      <NA>
12         2020       <NA>       <NA>         <NA>      <NA>      <NA>
13    

In [35]:
# Partial DataFrame's Analysis
analyze_df(df_lslist[['TStart', 'TStart_Acc', 'TStart_Qrt', 'TStop', 'TStop_Acc', 'TStop_Qrt']])

           data_type  #_unique  #_not_null  #_nulls  %_nulls
TStart        object       360         638       65     9.25
TStart_Acc    object         1          11      692    98.44
TStart_Qrt    object         0           0      703   100.00
TStop         object       314         556      147    20.91
TStop_Acc     object         1          19      684    97.30
TStop_Qrt     object         0           0      703   100.00 

Additionally:

The total number of null values is: 2994

The DataFrame has 703 rows and 6 columns.


In [36]:
# Data type conversion
df_lslist['Pars_TStart'] = df_lslist['TStart'].apply(vague_date_parser)
df_lslist['Pars_TStop'] = df_lslist['TStop'].apply(vague_date_parser)
# Extraction of date&time as string
df_lslist['Pars_TStart_str'] = df_lslist['Pars_TStart'].dt.strftime('%Y%m%d%H%M%S.%f').replace(np.nan, '')
df_lslist['Pars_TStop_str'] = df_lslist['Pars_TStop'].dt.strftime('%Y%m%d%H%M%S.%f').replace(np.nan, '')

# DataFrame's analysis
analyze_df(df_lslist)

                      data_type  #_unique  #_not_null  #_nulls  %_nulls
Site                     object       703         703        0     0.00
Code                     object         0           0      703   100.00
UCode                    object       669         703        0     0.00
Type                     object         3         703        0     0.00
StateCode                object        82         703        0     0.00
TStart                   object       360         638       65     9.25
TStop                    object       314         556      147    20.91
ShortName                object       512         703        0     0.00
Name                     object       699         703        0     0.00
Location                 object       112         336      367    52.20
Longitude                object       492         680       23     3.27
Latitude                 object       468         680       23     3.27
Error                   float64        30         703        0  

#### Working DataFrame

In [37]:
# DataFrame's sneak peek
df_lslist.head(1)

,Site,Code,UCode,Type,StateCode,TStart,TStop,ShortName,Name,Location,Longitude,Latitude,Error,Parent,ShortEName,EName,Group,UName,TStart_Acc,TStop_Acc,TStart_Qrt,TStop_Qrt,Pars_TStart,Pars_TStop,Pars_TStart_str,Pars_TStop_str
0,OCEAN,<NA>,OCEAN,LS,EARTH,<NA>,<NA>,Ocean,Ocean launch sites,<NA>,<NA>,<NA>,0.0,<NA>,Ocean,<NA>,<NA>,Ocean launch sites,<NA>,<NA>,<NA>,<NA>,NaT,NaT,,


In [38]:
# DataFrame backup
df_lslist_bkp = df_lslist.copy()

# Replacement of null values by empty string
# This is needed to avoid errors when writing into Google Sheets
df_lslist = df_lslist.where(pd.notnull(df_lslist), '')

# Removal of unnecessary columns
df_lslist = df_lslist.drop(
    columns = [
        'Code',
        'StateCode',
        'TStart',
        'TStop',
        'ShortEName',
        'EName',
        'Parent',
        'UName',
        'TStart_Acc',
        'TStop_Acc',
        'TStart_Qrt',
        'TStop_Qrt',
        'Pars_TStart',
        'Pars_TStop'
    ]
)

# DataFrame's analysis
analyze_df(df_lslist)

                data_type  #_unique  #_not_null  #_nulls  %_nulls
Site               object       703         703        0      0.0
UCode              object       669         703        0      0.0
Type               object         3         703        0      0.0
ShortName          object       512         703        0      0.0
Name               object       699         703        0      0.0
Location           object       113         703        0      0.0
Longitude          object       493         703        0      0.0
Latitude           object       469         703        0      0.0
Error             float64        30         703        0      0.0
Group              object        13         703        0      0.0
Pars_TStart_str    object       345         703        0      0.0
Pars_TStop_str     object       306         703        0      0.0 

Additionally:

The total number of null values is: 0

The DataFrame has 703 rows and 12 columns.


## Writing to Google Sheets

This section connects to a Google Sheets file, targets a specific set of tabs in that file, clears them, and then writes into them the data contained in the different, previously prepared `DataFrames`.

In [40]:
# Define the scope for Google Sheets and Google Drive
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]

# Load the credentials from the JSON file
creds_path = getpass.getpass("Credentials: ")  # Name of the credentials file
creds = Credentials.from_service_account_file(
    creds_path,
    scopes = scope
)

# Authenticate and create a client
client = gspread.authorize(creds)

Credenciales:  ········


In [41]:
# Open spreadsheets
sprdS_gcat = client.open('LOok__GCAT_ONLINE')
workS_ogList = sprdS_gcat.worksheet('2_Organizations')
workS_lsList = sprdS_gcat.worksheet('3_LaunchSites')
workS_laList = sprdS_gcat.worksheet('6_Launch20_25')

# Clean the worksheets
workS_ogList.clear()
workS_lsList.clear()
workS_laList.clear()

# Write from DataFrames to spreadsheets
## Organizations
workS_ogList.update(
    [df_oglist.columns.values.tolist()]
    + df_oglist.values.tolist()
)
## Launch Sites
workS_lsList.update(
    [df_lslist.columns.values.tolist()]
    + df_lslist.values.tolist() 
)
## Launches 2025
workS_laList.update(
    [df_lflist2000.columns.values.tolist()]
    + df_lflist2000.values.tolist() 
)

{'spreadsheetId': '17vMY17i-73Tc0MNMoS5nj1-n1ZCcG_cbcHhf2ZyGsbg',
 'updatedRange': "'6_Launch20_25'!A1:Y3961",
 'updatedRows': 3961,
 'updatedColumns': 25,
 'updatedCells': 99025}

A list of orbital rockets will be prepared to later insert the names into Looker Studio's filters.

In [42]:
# List by filtering for certain launch codes
orbitalR_list = df_lflist2000[df_lflist2000['LaunchCode'].str[0].isin(list('OTDY'))]['LV_Type'].unique()
# Prepare the rocket names for Looker Studio and print them
orbitalR_list = [value.replace(' ', '\ ') for value in orbitalR_list]
for value in orbitalR_list:
    print(value+',')

Atlas\ IIA,
Ariane\ 42L,
Chang\ Zheng\ 3A,
Minotaur\ I,
Soyuz-U-PVB,
Zenit-2,
Atlas\ IIAS,
Delta\ 7420-10C,
M-V,
Space\ Shuttle,
Proton-K/DM-2M,
Ariane\ 44LP,
Taurus\ 1110,
Zenit-3SL,
Ariane\ 5G,
Delta\ 7326-9.5,
Titan\ 402B/IUS,
Delta\ 7925-9.5,
Rokot,
Atlas\ 3A,
Proton-K/Briz-M,
Pegasus\ XL,
Chang\ Zheng\ 3,
Kosmos\ 11K65M,
Proton-K/DM-2,
Proton-K,
Titan\ 403B,
Delta\ 8930,
Chang\ Zheng\ 4B,
Ariane\ 44P,
Titan\ II\ SLV,
Dnepr,
Pegasus\ H,
Delta\ 7320-10,
Ariane\ 44L,
Start-1,
Tsiklon-3,
Chang\ Zheng\ 2F,
Titan\ 401B/Centaur,
Proton-M/Briz-M,
GSLV\ Mk\ I,
Soyuz-FG,
Delta\ 7425-10,
Molniya\ 8K78M,
H-IIA\ 202,
Taurus\ 2110,
Athena-1,
Titan\ 404B,
PSLV,
Delta\ 7920-10,
Tsiklon-2,
H-IIA\ 2024,
Delta\ 7920-10C,
Atlas\ 3B,
Ariane\ 42P,
Delta\ 7920-10L,
Shavit\ 1,
Delta\ 7425-9.5,
Proton-K/17S40,
Atlas\ V\ 401,
KT-1,
Delta\ 4M+(4,2),
Ariane\ 5ECA,
Delta\ 4M,
Spaceship\ One,
Delta\ 7925H,
Atlas\ V\ 521,
VLS-1,
Delta\ 7920H,
Chang\ Zheng\ 2D,
Strela,
Chang\ Zheng\ 2C,
Ariane\ 5G+,
Taurus\ 3210

<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
C:\Users\Juan Ignacio\AppData\Local\Temp\ipykernel_14568\1109266888.py:2: SyntaxWarning: invalid escape sequence '\ '
  orbitalR_list = [value.replace(' ', '\ ') for value in orbitalR_list]


A list of suborbital rockets will be prepared to later insert the names into Looker Studio's filters.

In [43]:
# List by filtering for launch code 'S'
suborbitalR_list = df_lflist2000[df_lflist2000['LaunchCode'].str[0].isin(list('S'))]['LV_Type'].unique()
# Prepare the rocket names for Looker Studio and print them
suborbitalR_list = [value.replace(' ', '\ ') for value in suborbitalR_list if value not in ['New Shepard', 'Spaceship Two', 'HASTE']]
suborbitalR_list = sorted(suborbitalR_list)
for value in suborbitalR_list:
    print(value+',')

ALV,
ATACMS,
ATV,
Aftershock,
Angara-1.2PP,
Ares\ I-X,
Astra\ Rocket\ 1.0,
Astra\ Rocket\ 2.0,
Atea-1,
Black\ Brant\ 10CM1,
Black\ Brant\ 8B,
Black\ Brant\ 8C,
Black\ Brant\ 9CM1,
Black\ Brant\ IX,
Black\ Brant\ VB,
Black\ Brant\ VC,
Black\ Brant\ X,
Black\ Brant\ XI,
Black\ Brant\ XIA,
Black\ Brant\ XII,
Black\ Brant\ XIIA,
Black\ Dagger,
CSXT,
Centenario,
DART,
DF-21,
Duqm-1,
Evo\ Space,
Excalibur,
FalconLaunch,
GSLV\ Mk\ III,
Ghadr\ 1,
Gradicom\ 1,
Gradicom\ 2,
Gun\ Projectile\ 5-inch,
HANBIT-TLV,
HS9,
HYSR,
Hapith\ I,
Huayi-1,
Hwasong-12A,
ILR-33\ Bursztyn\ 2K,
IM/IM,
IM/IO,
Icarus,
Improved\ Malemute,
Improved\ Orion,
KSLV-II\ TLV,
KSR-III,
Kavoshgar\ K110,
Kavoshgar\ M5,
Kingfisher,
Kuaizhou,
M-100B,
MESOS,
MN-300,
MR-30,
MT-135,
MT-135P,
Maxus,
Meraki,
Mesquito,
Miura\ 1,
Momo,
N2ORTH,
Nazeat\ 6H,
Nike\ Improved\ Orion,
Nike\ Orion,
Nucleus,
OS-X0,
OS-X1,
OS-X6B,
Oriole\ IIIA,
Oriole\ IV,
Orion,
Pathfinder\ Zombie,
RH-200,
RH-200SV,
RH-300\ Mk\ II,
RH-560/300\ Mk\ II,
RH-560\ Mk

<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
C:\Users\Juan Ignacio\AppData\Local\Temp\ipykernel_14568\2252196468.py:2: SyntaxWarning: invalid escape sequence '\ '
  suborbitalR_list = [value.replace(' ', '\ ') for value in suborbitalR_list if value not in ['New Shepard', 'Spaceship Two', 'HASTE']]
